# Deploying Machine Learning Models Final Project

I decided to use the amazon Musical Instruments data set for this project. I feel like it has a good variaty of data and enough points to get a good analysis and recommender system from.

I will be creating a similarity bases recommender system for prating prediction.

In [1]:
import gzip
from collections import defaultdict
import random
import numpy
import scipy.optimize

In [2]:
path = 'amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz'

In [3]:
f = gzip.open(path, 'rt', encoding='utf8')

In [4]:
header = f.readline()
header = header.strip().split('\t')

In [5]:
dataset = []

In [6]:
# parsing through the data set
for line in f:
    fields = line.strip().split('\t')
    d = dict(zip(header, fields))
    d['star_rating'] = int(d['star_rating'])
    d['helpful_votes'] = int(d['helpful_votes'])
    d['total_votes'] = int(d['total_votes'])
    dataset.append(d)  

EOFError: Compressed file ended before the end-of-stream marker was reached

In [7]:
# creating some data structures that will be useful in the analysis
reviewsPerUser = defaultdict(list)
reviewsPerItem = defaultdict(list)
usersPerItem = defaultdict(set)
itemsPerUser = defaultdict(set)

In [8]:
itemNames = {}
for d in dataset:
    user,item = d['customer_id'], d['product_id']
    usersPerItem[item].add(user)
    itemsPerUser[user].add(item)
    itemNames[item] = d['product_title']

In [9]:
# adding users to the user data set and items to the item dataset
for d in dataset:
    user,item = d['customer_id'], d['product_id']
    reviewsPerUser[user].append(d)
    reviewsPerItem[item].append(d)

In [10]:
# find the mean star rating of all the ratings
ratingMean = sum([d['star_rating'] for d in dataset]) / len(dataset)

In [11]:
# a function to calculate the Jaccard value
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    return numer / denom

In [12]:
# creating the predictor function
def predictRating(user, item):
    ratings = []
    similarities = []
    for d in reviewsPerUser[user]:
        i2 = d['product_id']
        if i2 == item: continue
        ratings.append(d['star_rating'])
        similarities.append(Jaccard(usersPerItem[item], usersPerItem[i2]))
    if (sum(similarities) > 0):
        weightedRatings = [(x*y) for x,y in zip(ratings,similarities)]
        return sum(weightedRatings) / sum(similarities)
    else:
        # User hasn't rated any similar items
        return ratingMean

In [13]:
u,i = dataset[1]['customer_id'], dataset[1]['product_id']

In [14]:
predictRating(u,i)

5.0

In [15]:
# creating a function to calculate mean square error
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions, labels)]
    return sum(differences) / len(differences)

In [16]:
# creating an array of all mean predictions
alwasyPredictMean = [ratingMean for d in dataset]

In [17]:
cfPredictions = [predictRating(d['customer_id'], d['product_id']) for d in dataset]

In [18]:
labels = [d['star_rating'] for d in dataset]

In [19]:
MSE(alwasyPredictMean, labels)

1.45376606289067

In [20]:
MSE(cfPredictions, labels)

1.5639964910086366

It turns out here that our predictor system is a little worse than if we were to just predict the mean value every time. 